In [151]:
import pandas as pd
import tkinter as tkr
from tkinter import *

In [152]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [153]:
movies = pd.read_csv('movies.csv')
movies = movies.sort_values(by='title')

In [154]:
ratings = pd.read_csv('ratings.csv')
ratings


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [155]:
mov_rat = pd.merge(ratings,movies, on='movieId')

In [156]:
mov_rat

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


In [157]:
pivot = mov_rat.pivot_table(index='userId',columns='title',values='rating')

In [158]:
pivot.sample(10)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
app = tkr.Tk('Movie recommender')
app.title('Movie Recommender')
app.geometry('600x400')

''

In [160]:
w = Scrollbar(app)

In [161]:
myList = Listbox(app,width=50,yscrollcommand=w.set)
for i in movies['title']:
    myList.insert(END,i)
myList.pack(side='left',fill= BOTH)
w.config(command = myList.yview)
w.place(in_ =myList,relx=1,relheight=1)


In [162]:
def mov_recc():
    var = myList.get(myList.curselection())
    simm_corr = pivot.corrwith(pivot[var])
    simm_corr = pd.DataFrame(simm_corr,columns=['similarity'])
    simm_corr['count'] = mov_rat.groupby('title')['rating'].count()
    cc = simm_corr[simm_corr['count']>30].sort_values(by = 'similarity',ascending=False).head(5)
    cc_list = Listbox(app,width=30)
    sim_list = [str(i) for i in (cc[cc.columns[0]])]
    if sim_list[0] == 'nan':
        cc_list.insert(END,'please select another movie')
    else:
        for i in cc.index[:3].tolist():
            cc_list.insert(END,i)
    msg = tkr.Variable(app)
    msg.set('Recommendation:')
    tkr.Label(app,textvariable = msg).place(x=320,y=30)
    cc_list.place(x=320,y=60)
    print(var,cc,sep='\n')
    

In [163]:
tkr.Button(app,text=' show Recommendation', command= mov_recc).place(x=390,y=350)
app.mainloop()

C:\Users\anshi\anaconda3\lib\site-packages\numpy\lib\function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\anshi\anaconda3\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


10 Cent Pistol (2015)
                                                    similarity  count
title                                                                
Lethal Weapon 2 (1989)                                     1.0     47
Godfather: Part III, The (1990)                            1.0     45
Pulp Fiction (1994)                                        1.0    307
Peter Pan (1953)                                           1.0     41
Naked Gun: From the Files of Police Squad!, The...         1.0     42
